In [9]:
import pyodbc
from pyodk.client import Client  # ODK central library
import pandas as pd  # Dataframe manipulation library
from sqlalchemy import create_engine, inspect #SQL database library
import db_config #python file that stores db connection details
import project_config #python file consisting of project IDs and form IDs

In [10]:
# SQL Server connection details from db_config module
server = db_config.server
database = db_config.database
username = db_config.username
password = db_config.password
driver = db_config.driver

In [11]:
# Establish a connection to the SQL Server
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={connection_string}")

In [12]:
# Set up the ODK client
client = Client(config_path="config.toml", cache_path="cache.toml")
client.open()

In [13]:
# Query the project_id and form_id values from the database
query1 = "SELECT project_id, form_ids, status FROM OdkCentralFormIds"  # Replace YourTableName with the actual table name
# Fetch data using pandas DataFrame
project_form_data = pd.read_sql(query1, engine)
project_form_data

,project_id,form_ids,status
0,3,assess_improve_household_individual_crf,active
1,3,optimal_malaria_microscopy_results_crf,active
2,3,optimal_lab_request_result_crf,active
3,3,optimal_drug_admin_crf,active
4,3,optimal_visits_crf,active
5,3,optimal_pk_sampling_crf,active
6,3,optimal_capillary_hb_crf,active
7,3,optimal_screening_enrolment_crf,active
8,3,optimal_study_meds_crf,active
9,3,optimal_ae_sae_crf,active


In [14]:
for index, row in project_form_data.iterrows():
    default_project_id = int(row['project_id'])
    form_ids = row['form_ids'].split(',')  # Split the comma-separated form IDs
    
    for form_id in form_ids:
        try:
            client.default_project_id = default_project_id
            client.submissions.list(form_id=form_id)
            json = client.submissions.get_table(form_id=form_id)['value']
            df = pd.json_normalize(json, sep='-')
            df = df.set_index('__id')
            
            # Check if the form's status is "active"
            form_status = row['status']  # Assuming the status column name is 'status'
            
            if form_status.lower() == 'active':
                # Process the dataframe (df) for each form ID
                df.columns = [col.split('-')[-1] for col in df.columns]
                df_filtered = df.loc[:, ~df.columns.str.startswith('generated_note')]
                df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.startswith('reserved_name_for_field_list_labels_')]
                df_filtered = df_filtered.drop(columns=['deviceId'])
                
                # Check if the table already exists
                table_name = form_id.replace('-', '_')
                inspector = inspect(engine)
                table_exists = inspector.has_table(table_name)
                
                if table_exists:
                    # Append new records by cross-checking instanceID
                    existing_records = pd.read_sql(f"SELECT instanceID FROM {table_name}", con=engine)
                    new_records = df_filtered[~df_filtered['instanceID'].isin(existing_records['instanceID'])]
                    new_records.to_sql(table_name, con=engine, if_exists='append', index=False)
                    print(f"APPENDED new records for form_id: {form_id}. SQL table name: {table_name}")
                else:
                    # Create a new table
                    df_filtered.to_sql(table_name, con=engine, if_exists='replace', index=False)
                    print(f"Created new table for form_id: {form_id}. SQL table name: {table_name}")
            else:
                print(f"Form {form_id} is inactive. Skipping extraction.")
                
        except Exception as e:
            # Print the message indicating extraction failure
            print(f"Extraction failed for form_id: {form_id}. Error: {str(e)}")

# Print the message indicating extraction completion
print("Extraction process completed!")


APPENDED new records for form_id: assess_improve_household_individual_crf. SQL table name: assess_improve_household_individual_crf
APPENDED new records for form_id: optimal_malaria_microscopy_results_crf. SQL table name: optimal_malaria_microscopy_results_crf
APPENDED new records for form_id: optimal_lab_request_result_crf. SQL table name: optimal_lab_request_result_crf
APPENDED new records for form_id: optimal_drug_admin_crf. SQL table name: optimal_drug_admin_crf
APPENDED new records for form_id: optimal_visits_crf. SQL table name: optimal_visits_crf
APPENDED new records for form_id: optimal_pk_sampling_crf. SQL table name: optimal_pk_sampling_crf
APPENDED new records for form_id: optimal_capillary_hb_crf. SQL table name: optimal_capillary_hb_crf
APPENDED new records for form_id: optimal_screening_enrolment_crf. SQL table name: optimal_screening_enrolment_crf
APPENDED new records for form_id: optimal_study_meds_crf. SQL table name: optimal_study_meds_crf
APPENDED new records for form